# 환경설정

In [19]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager, rc
%matplotlib inline
# from functools import reduce

# set font
font_name = font_manager.FontProperties(
    fname='c:/Windows/Fonts/malgun.ttf').get_name()
rc('font', family=font_name)
# matplotlib.rcParams['axes.unicode_minus'] = False
sns.set(font=font_name,
        rc={'axes.unicode_minus': False})  # to show negative sign on graphs

# set option
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.precision', 2)


# 우편번호

In [2]:
address = pd.read_csv('./data/raw/우편번호_21.csv', encoding='cp949')
# extract '도이름', '시군구이름'
address = address.groupby(['도이름', '시군구이름']).size().reset_index().iloc[:,:2]
# rename
address = address.rename(columns={'도이름': '시도명', '시군구이름': '시군구명'})
# processing 세종시
address.loc[address['시도명'] == '세종특별자치시', '시군구명'] = '세종시'
# save
address.to_csv('./data/processed/address.csv', encoding='cp949', index=False)


In [3]:
check1 = set()
check2 = set()
# check by address
def check(dataframe):
    check_address = address['시도명']+' '+address['시군구명']
    check_dataframe = dataframe['시도명']+' '+dataframe['시군구명']
    result1 = set(check_address) - set(check_dataframe)
    check1.update(result1)
    result2 = set(check_dataframe) - set(check_address)
    check2.update(result2)
    return print(f'result1\n{result1}\n\nresult2\n{result2}')


# 건강생활실천율

In [4]:
# combine first two rows
healthy_life = pd.read_csv(
    './data/raw/건강생활실천율_17-20.csv', header=[0, 1], encoding='cp949')
healthy_life.columns = healthy_life.columns.map('_'.join)
# feature selection
columns = ['시군구별(1)_시군구별(1)', '시군구별(2)_시군구별(2)', '시군구별(3)_시군구별(3)',
           '2017_조율 (%)', '2018_조율 (%)', '2019_조율 (%)', '2020_조율 (%)']
healthy_life = healthy_life[columns]
# drop '소계' in (2)
condition = (healthy_life['시군구별(2)_시군구별(2)'].str.contains('소계') == False)
healthy_life = healthy_life.loc[condition]
# 소계 replace with '' in (3)
healthy_life['시군구별(3)_시군구별(3)'] = healthy_life['시군구별(3)_시군구별(3)'].str.replace(
    '소계', '')
# combine 시군구 (2) and (3)
healthy_life['시군구별(2)_시군구별(2)'] = healthy_life['시군구별(2)_시군구별(2)'] + ' ' + \
    healthy_life['시군구별(3)_시군구별(3)']
healthy_life = healthy_life.drop('시군구별(3)_시군구별(3)', axis='columns')
# rename
healthy_life = healthy_life.rename(
    columns={'시군구별(1)_시군구별(1)': '시도명', '시군구별(2)_시군구별(2)': '시군구명'})
# rstrip
healthy_life['시군구명'] = healthy_life['시군구명'].str.rstrip()
# save
healthy_life.to_csv('./data/processed/healthy_life.csv', encoding='cp949', index=False)


In [5]:
check(healthy_life)


result1
{'경기도 여주군', '경상남도 창원시 진해구', '충청남도 천안시 서북구', '인천광역시 남구', '전라북도 전주시 완산구', '경상남도 창원시 성산구', '충청북도 청원군', '경상남도 창원시 의창구', '전라북도 전주시 덕진구', '충청남도 천안시 동남구', '경상남도 창원시 마산합포구', '경상남도 창원시 마산회원구'}

result2
{'인천광역시 미추홀구', '경기도 남양주시 남양주 풍양', '경기도 평택시 송탄', '제주특별자치도 제주시 제주', '경기도 수원시', '경상북도 구미시 구미', '경상남도 창원시 진해', '경기도 고양시', '경상북도 포항시', '경상북도 구미시 선산', '충청북도 청주시 청원구', '제주특별자치도 서귀포시 서부', '제주특별자치도 제주시 서부', '경상남도 창원시', '경상남도 창원시 마산', '경기도 안양시', '경상남도 창원시 창원', '경기도 용인시', '제주특별자치도 서귀포시 동부', '경기도 남양주시 남양주', '경기도 안산시', '경기도 부천시', '경기도 평택시 평택', '제주특별자치도 서귀포시 서귀포', '제주특별자치도 제주시 동부', '경기도 성남시', '충청남도 천안시', '전라북도 전주시', '충청북도 청주시', '충청북도 청주시 서원구', '경기도 여주시'}


# 합계출산율

In [6]:
# use Excel to preprocess
childbirth = pd.read_csv('./data/raw/합계출산율_17-20.csv', encoding='cp949')
# save
childbirth.to_csv('./data/processed/childbirth.csv',
                  encoding='cp949', index=False)


In [7]:
check(childbirth)


result1
{'경기도 성남시 수정구', '충청북도 청주시 흥덕구', '경기도 수원시 권선구', '경상남도 창원시 성산구', '경상남도 창원시 의창구', '경상남도 창원시 마산합포구', '경기도 안양시 동안구', '경상남도 창원시 마산회원구', '경기도 용인시 수지구', '경기도 고양시 일산서구', '충청남도 천안시 서북구', '전라북도 전주시 완산구', '경상북도 포항시 북구', '경기도 고양시 덕양구', '경기도 안산시 단원구', '전라북도 전주시 덕진구', '경기도 부천시 소사구', '경상북도 포항시 남구', '경상남도 창원시 진해구', '경기도 수원시 팔달구', '경기도 부천시 원미구', '경기도 용인시 기흥구', '경기도 고양시 일산동구', '경기도 부천시 오정구', '경기도 안양시 만안구', '경기도 안산시 상록구', '경기도 용인시 처인구', '경기도 수원시 영통구', '경기도 수원시 장안구', '경기도 성남시 중원구', '충청남도 천안시 동남구', '경기도 성남시 분당구', '충청북도 청주시 상당구'}

result2
{'인천광역시 미추홀구', '경기도 중원구', '충청남도 서북구', '경기도 포천군', '충청북도 서원구', '경기도 수정구', '경기도 원미구', '제주특별자치도 남제주군', '제주특별자치도 북제주군', '경상남도 회원구', '경기도 덕양구', '경기도 단원구', '경기도 수원시', '경기도 분당구', '경상남도 의창구', '경상남도 마산회원구', '경기도 광주군', '전라북도 덕진구', '경기도 고양시', '경상남도 마산시', '경기도 장안구', '경상북도 포항시', '전라북도 완산구', '경기도 상록구', '경기도 만안구', '충청남도 동남구', '경기도 일산서구', '경상북도 북구', '경상남도 창원시', '충청남도 당진군', '경상북도 남구', '충청북도 청원구', '경기도 안양시', '경기도 양주군', '경기도 팔달구', '경기도 용인시', '경기도 기흥구', '경기도 영통구', '경상남도 진해시', '경기도 오정구',

# 인구증가율

In [8]:
# use Excel to preprocess
# combine first two rows
population_growth = pd.read_csv(
    './data/raw/인구증가율_17-20.csv', header=[0, 1], encoding='cp949')
population_growth.columns = population_growth.columns.map('_'.join)
# feature selection
columns = ['시도명_Unnamed: 0_level_1', '시군구명_행정구역별', '2017_인구증가율<br>{(A-B)÷B}×100 (%)',
           '2018_인구증가율<br>{(A-B)÷B}×100 (%)', '2019_인구증가율<br>{(A-B)÷B}×100 (%)',
           '2020_인구증가율<br>{(A-B)÷B}×100 (%)']
population_growth = population_growth[columns]
# rename
population_growth = population_growth.rename(
    columns={'시도명_Unnamed: 0_level_1': '시도명', '시군구명_행정구역별': '시군구명'})
# save
population_growth.to_csv(
    './data/processed/population_growth.csv', encoding='cp949', index=False)


In [9]:
check(population_growth)


result1
{'제주특별자치도 서귀포시', '경기도 성남시 수정구', '충청북도 청주시 흥덕구', '경기도 수원시 권선구', '경상남도 창원시 성산구', '경상남도 창원시 의창구', '경상남도 창원시 마산합포구', '경기도 안양시 동안구', '경상남도 창원시 마산회원구', '제주특별자치도 제주시', '경기도 용인시 수지구', '경기도 고양시 일산서구', '충청남도 천안시 서북구', '전라북도 전주시 완산구', '경상북도 포항시 북구', '경기도 고양시 덕양구', '충청북도 청주시 상당구', '경기도 안산시 단원구', '세종특별자치시 세종시', '전라북도 전주시 덕진구', '경기도 부천시 소사구', '경상북도 포항시 남구', '경상남도 창원시 진해구', '경기도 수원시 팔달구', '경기도 부천시 원미구', '경기도 용인시 기흥구', '경기도 고양시 일산동구', '경기도 부천시 오정구', '경기도 안양시 만안구', '경기도 안산시 상록구', '경기도 여주군', '경기도 수원시 영통구', '경기도 용인시 처인구', '경기도 수원시 장안구', '경기도 성남시 중원구', '충청남도 천안시 동남구', '경기도 성남시 분당구', '충청북도 청원군'}

result2
{'경상남도 창원시', '인천광역시 미추홀구', '경기도 안산시', '경기도 고양시', '경기도 부천시', '경상북도 포항시', '경기도 안양시', '경기도 용인시', '경기도 수원시', '경기도 성남시', '세종특별자치시 세종특별자치시', '충청남도 천안시', '전라북도 전주시', '제주특별자치도 제주 제주시', '충청북도 청주시', '제주특별자치도 제주 서귀포시', '경기도 여주시'}


# 노인천명당_노인여가복지시설수

In [10]:
welfare_facility = pd.read_csv(
    './data/raw/노인천명당_노인여가복지시설수_17-20.csv', header=[0, 1], encoding='cp949')
welfare_facility.columns = welfare_facility.columns.map('_'.join)
columns = ['행정구역별(1)_행정구역별(1)', '행정구역별(2)_행정구역별(2)', '2017_노인천명당 노인여가복지시설수<br>(A÷B×1000) (개)',
           '2018_노인천명당 노인여가복지시설수<br>(A÷B×1000) (개)', '2019_노인천명당 노인여가복지시설수<br>(A÷B×1000) (개)',
           '2020_노인천명당 노인여가복지시설수<br>(A÷B×1000) (개)']
welfare_facility = welfare_facility[columns]
# drop '소계'
condition = (welfare_facility['행정구역별(2)_행정구역별(2)'].str.contains('소계') == False)
welfare_facility = welfare_facility.loc[condition]
# rename
welfare_facility = welfare_facility.rename(
    columns={'행정구역별(1)_행정구역별(1)': '시도명', '행정구역별(2)_행정구역별(2)': '시군구명'})
# save
welfare_facility.to_csv(
    './data/processed/welfare_facility.csv', encoding='cp949', index=False)


In [11]:
check(welfare_facility)


result1
{'경기도 성남시 수정구', '충청북도 청주시 흥덕구', '경기도 수원시 권선구', '경상남도 창원시 성산구', '경상남도 창원시 의창구', '경상남도 창원시 마산합포구', '경기도 안양시 동안구', '경상남도 창원시 마산회원구', '경기도 용인시 수지구', '경기도 고양시 일산서구', '충청남도 천안시 서북구', '전라북도 전주시 완산구', '경상북도 포항시 북구', '경기도 고양시 덕양구', '충청북도 청주시 상당구', '경기도 안산시 단원구', '세종특별자치시 세종시', '전라북도 전주시 덕진구', '경기도 부천시 소사구', '경상북도 포항시 남구', '경상남도 창원시 진해구', '경기도 수원시 팔달구', '경기도 부천시 원미구', '경기도 용인시 기흥구', '경기도 고양시 일산동구', '경기도 부천시 오정구', '경기도 안양시 만안구', '경기도 안산시 상록구', '경기도 여주군', '경기도 수원시 영통구', '경기도 용인시 처인구', '경기도 수원시 장안구', '경기도 성남시 중원구', '충청남도 천안시 동남구', '경기도 성남시 분당구', '충청북도 청원군'}

result2
{'경상남도 창원시', '인천광역시 미추홀구', '경기도 고양시', '경기도 부천시', '경상북도 포항시', '경기도 안양시', '경기도 용인시', '경기도 수원시', '경기도 성남시', '충청남도 천안시', '전라북도 전주시', '경기도 안산시', '충청북도 청주시', '경기도 여주시'}


# 고령인구비율

In [12]:
elderly_ratio = pd.read_csv(
    './data/raw/고령인구비율_17-21.csv', header=[0, 1], encoding='cp949')
elderly_ratio.columns = elderly_ratio.columns.map('_'.join)
columns = ['행정구역별(1)_행정구역별(1)', '행정구역별(2)_행정구역별(2)',
           '2017_고령인구비율<br>(A÷B×100) (%)', '2018_고령인구비율<br>(A÷B×100) (%)',
           '2019_고령인구비율<br>(A÷B×100) (%)', '2020_고령인구비율<br>(A÷B×100) (%)',
           '2021_고령인구비율<br>(A÷B×100) (%)']
elderly_ratio = elderly_ratio[columns]
# drop '소계'
condition = (elderly_ratio['행정구역별(2)_행정구역별(2)'].str.contains('소계') == False)
elderly_ratio = elderly_ratio.loc[condition]
# rename
elderly_ratio = elderly_ratio.rename(
    columns={'행정구역별(1)_행정구역별(1)': '시도명', '행정구역별(2)_행정구역별(2)': '시군구명'})
# save
elderly_ratio.to_csv('./data/processed/elderly_ratio.csv',
                     encoding='cp949', index=False)


In [13]:
check(elderly_ratio)

result1
{'경기도 성남시 수정구', '충청북도 청주시 흥덕구', '경기도 수원시 권선구', '경상남도 창원시 성산구', '경상남도 창원시 의창구', '경상남도 창원시 마산합포구', '경기도 안양시 동안구', '경상남도 창원시 마산회원구', '경기도 용인시 수지구', '경기도 고양시 일산서구', '충청남도 천안시 서북구', '전라북도 전주시 완산구', '경상북도 포항시 북구', '경기도 고양시 덕양구', '충청북도 청주시 상당구', '경기도 안산시 단원구', '세종특별자치시 세종시', '전라북도 전주시 덕진구', '경기도 부천시 소사구', '경상북도 포항시 남구', '경상남도 창원시 진해구', '경기도 수원시 팔달구', '경기도 부천시 원미구', '경기도 용인시 기흥구', '경기도 고양시 일산동구', '경기도 부천시 오정구', '경기도 안양시 만안구', '경기도 안산시 상록구', '경기도 여주군', '경기도 수원시 영통구', '경기도 용인시 처인구', '경기도 수원시 장안구', '경기도 성남시 중원구', '충청남도 천안시 동남구', '경기도 성남시 분당구', '충청북도 청원군'}

result2
{'경상남도 창원시', '인천광역시 미추홀구', '경기도 고양시', '경기도 부천시', '경상북도 포항시', '경기도 안양시', '경기도 용인시', '경기도 수원시', '경기도 성남시', '충청남도 천안시', '전라북도 전주시', '경기도 안산시', '충청북도 청주시', '경기도 여주시'}


# 경제활동인구
- 시도 level
- 연령별

In [14]:
econ_active = pd.read_csv(
    './data/raw/연령별_경제활동인구_17-21.csv', header=[0, 1], encoding='cp949')
# combine first two rows
econ_active.columns = econ_active.columns.map('_'.join)
columns = ['시도별_시도별', '연령계층별_연령계층별', '2017_경제활동참가율 (%)', '2018_경제활동참가율 (%)',
           '2019_경제활동참가율 (%)', '2020_경제활동참가율 (%)', '2021_경제활동참가율 (%)']
econ_active = econ_active[columns]
# rearrange table
econ_active = econ_active.pivot(index='시도별_시도별', columns='연령계층별_연령계층별').reset_index()
# combine first two rows
econ_active.columns = econ_active.columns.map('_'.join)
econ_active = econ_active.rename(columns={'시도별_시도별_':'시도명'})
# save
econ_active.to_csv('./data/processed/econ_active.csv',
                   encoding='cp949', index=False)


# 상권관련
- 골프용품판매
- 실내골프연습장
- 비만/피부관리
- 커피전문점/카페/다방
- 학원-어린이영어
- 어린이집
- 장의사/관련서비스-종합

In [15]:
path = 'data/raw/소상공인시장진흥공단_상가(상권)정보_20211231'
# ut file in files
files = []
for file in os.listdir(path):
    if file.endswith('.csv'):
        file = os.path.join(
            path, file)
        files.append(file)
# merge
left = pd.DataFrame()
for file in files:
    right = pd.read_csv(file, low_memory=False)
    left = pd.concat([left, right], axis=0, ignore_index=True)


In [16]:
check(left)

result1
{'경기도 여주군', '인천광역시 남구', '경기도 부천시 원미구', '세종특별자치시 세종시', '경기도 부천시 소사구', '경기도 부천시 오정구', '충청북도 청원군'}

result2
{'경상남도 창원시', '인천광역시 미추홀구', '경기도 고양시', '경기도 부천시', '경기도 용인시', '경기도 수원시', '충청남도 홍북읍', '충청북도 청주시 청원구', '세종특별자치시 세종특별자치시', '전라북도 전주시', '경기도 안산시', '충청북도 청주시 서원구', '경기도 여주시'}


In [17]:
# feature selection
columns = ['시도명', '시군구명', '상권업종소분류명']
left = left[columns]
# 골프용품판매
golf_shop = left[left['상권업종소분류명'] == '골프용품판매']
golf_shop = golf_shop.groupby(['시도명', '시군구명']).size().reset_index()
golf_shop.to_csv('./data/processed/golf_shop.csv',
                 encoding='cp949', index=False)
# 실내골프연습장
driving_range = left[left['상권업종소분류명'] == '실내골프연습장']
driving_range = driving_range.groupby(['시도명', '시군구명']).size().reset_index()
driving_range.to_csv('./data/processed/driving_range.csv', encoding='cp949', index=False)
# 비만/피부관리
obese_manage = left[left['상권업종소분류명'] == '비만/피부관리']
obese_manage = obese_manage.groupby(['시도명', '시군구명']).size().reset_index()
obese_manage.to_csv('./data/processed/obese_manage.csv',
                    encoding='cp949', index=False)
# 커피전문점/카페/다방
coffee = left[left['상권업종소분류명'] == '커피전문점/카페/다방']
coffee = coffee.groupby(['시도명', '시군구명']).size().reset_index()
coffee.to_csv('./data/processed/coffee.csv', encoding='cp949', index=False)
# 학원-어린이영어
kids_eng = left[left['상권업종소분류명'] == '학원-어린이영어']
kids_eng = kids_eng.groupby(['시도명', '시군구명']).size().reset_index()
kids_eng.to_csv('./data/processed/kids_eng.csv', encoding='cp949', index=False)
# 어린이집
day_care = left[left['상권업종소분류명'] == '어린이집']
day_care = day_care.groupby(['시도명', '시군구명']).size().reset_index()
day_care.to_csv('./data/processed/day_care.csv', encoding='cp949', index=False)
# 장의사/관련서비스-종합
undertaker = left[left['상권업종소분류명'] == '장의사/관련서비스-종합']
undertaker = undertaker.groupby(['시도명', '시군구명']).size().reset_index()
undertaker.to_csv('./data/processed/undertaker.csv',
                  encoding='cp949', index=False)


# TBD

In [18]:
check1

{'경기도 고양시 덕양구',
 '경기도 고양시 일산동구',
 '경기도 고양시 일산서구',
 '경기도 부천시 소사구',
 '경기도 부천시 오정구',
 '경기도 부천시 원미구',
 '경기도 성남시 분당구',
 '경기도 성남시 수정구',
 '경기도 성남시 중원구',
 '경기도 수원시 권선구',
 '경기도 수원시 영통구',
 '경기도 수원시 장안구',
 '경기도 수원시 팔달구',
 '경기도 안산시 단원구',
 '경기도 안산시 상록구',
 '경기도 안양시 동안구',
 '경기도 안양시 만안구',
 '경기도 여주군',
 '경기도 용인시 기흥구',
 '경기도 용인시 수지구',
 '경기도 용인시 처인구',
 '경상남도 창원시 마산합포구',
 '경상남도 창원시 마산회원구',
 '경상남도 창원시 성산구',
 '경상남도 창원시 의창구',
 '경상남도 창원시 진해구',
 '경상북도 포항시 남구',
 '경상북도 포항시 북구',
 '세종특별자치시 세종시',
 '인천광역시 남구',
 '전라북도 전주시 덕진구',
 '전라북도 전주시 완산구',
 '제주특별자치도 서귀포시',
 '제주특별자치도 제주시',
 '충청남도 천안시 동남구',
 '충청남도 천안시 서북구',
 '충청북도 청원군',
 '충청북도 청주시 상당구',
 '충청북도 청주시 흥덕구'}

In [63]:
check2

{'경기도 고양시',
 '경기도 광주군',
 '경기도 권선구',
 '경기도 기흥구',
 '경기도 남양주시 남양주',
 '경기도 남양주시 남양주 풍양',
 '경기도 단원구',
 '경기도 덕양구',
 '경기도 동안구',
 '경기도 만안구',
 '경기도 부천시',
 '경기도 분당구',
 '경기도 상록구',
 '경기도 성남시',
 '경기도 소사구',
 '경기도 수원시',
 '경기도 수정구',
 '경기도 수지구',
 '경기도 안산시',
 '경기도 안양시',
 '경기도 양주군',
 '경기도 여주시',
 '경기도 영통구',
 '경기도 오정구',
 '경기도 용인시',
 '경기도 원미구',
 '경기도 일산구',
 '경기도 일산동구',
 '경기도 일산서구',
 '경기도 장안구',
 '경기도 중원구',
 '경기도 처인구',
 '경기도 팔달구',
 '경기도 평택시 송탄',
 '경기도 평택시 평택',
 '경기도 포천군',
 '경기도 화성군',
 '경상남도 마산시',
 '경상남도 마산합포구',
 '경상남도 마산회원구',
 '경상남도 성산구',
 '경상남도 의창구',
 '경상남도 진해구',
 '경상남도 진해시',
 '경상남도 창원시',
 '경상남도 창원시 마산',
 '경상남도 창원시 진해',
 '경상남도 창원시 창원',
 '경상남도 합포구',
 '경상남도 회원구',
 '경상북도 구미시 구미',
 '경상북도 구미시 선산',
 '경상북도 남구',
 '경상북도 북구',
 '경상북도 포항시',
 '세종특별자치시 세종특별자치시',
 '인천광역시 미추홀구',
 '전라북도 덕진구',
 '전라북도 완산구',
 '전라북도 전주시',
 '제주특별자치도 남제주군',
 '제주특별자치도 북제주군',
 '제주특별자치도 서귀포시 동부',
 '제주특별자치도 서귀포시 서귀포',
 '제주특별자치도 서귀포시 서부',
 '제주특별자치도 제주 서귀포시',
 '제주특별자치도 제주 제주시',
 '제주특별자치도 제주시 동부',
 '제주특별자치도 제주시 서부',
 '제주특별자치도 제주시 제주',
 '충청남도 당진군',
 '충청남도